In [2]:
# ! pip install tensorflow --upgrade

Content
- Get to Know TensorFlow2.x
    - [History between Keras and TensorFlow](#History-between-Keras-and-TensorFlow:)
    - [Keras and tf.keras](#Keras-and-tf.keras)
- [Eager Execution](#Eager-Execution)
    - [Eager execution](#Eager-execution)
    - [Graph execution](#Graph-execution)
    - [Why TensorFlow adopted Eager Execution?](#Why-TensorFlow-adopted-Eager-Execution?)
- [Tensors](#Tensors)
    - [Create tensors](#Create-tensors)
    - [indexing](#indexing)
    - [Operations with tensors](#Operations-with-tensors)
- [Variables](#Variables)
    - [Operations with variables](#Operations-with-variables)
    - [Extend](#Extend)
    - [Hardware selection for variables](#Hardware-selection-for-variables)

**TensorFlow** is an end-to-end framework and platform designed to build and train machine learning models, especially deep learning models.

**Keras** is a deep learning API written in Python, running on top of the machine learning platform TensorFlow.

## History between Keras and TensorFlow:

- Keras was first developed by Francois Chollet for his own reserach in March 27th, 2015. Back then, TensorFlow hasn't been developed. There weren't too many deep learning libraries available - the popular ones included Torch, Theano, and Caffe. 
- In order to train the neural networks, Keras required a **backend** (computational engine). Keras's default backend was Theano until v1.1.0. At the same time, Google released TensorFlow. Keras started supporting it as a backend, and became the default starting from the release of Keras v1.1.0.
     - Keras started making TensorFlow default due to its popularity; and TensorFlow users were also becoming increasingly drawn to the simplicity of the high-level Keras API.
- Google announced TensorFlow 2.0 in June 2019, they declared that Keras is now the official high-level API of TensorFlow.

[Link](https://www.pyimagesearch.com/2019/10/21/keras-vs-tf-keras-whats-the-difference-in-tensorflow-2-0/)

In [3]:
import tensorflow as tf
import numpy as np
import keras

In [4]:
print(tf.__version__)

2.4.1


## Keras and tf.keras

- tf.keras submodule was introduced in TensorFlow v1.10.0, integrating Keras directly within TensorFlow package itself.
- Release of Keras 2.3.0: 
     - It is the first release of Keras that brings the Keras package in sync with tf.keras
     - It is the final release of Keras that will support multiple backends 
     - The keras package will only support bug fixes. Developers should start using tf.keras

In [5]:
tf.keras == keras

False

In [6]:
tf.keras.Sequential == keras.models.Sequential

True

In [7]:
tf.keras.models.Model == tf.keras.Model

True

# Eager Execution

https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6


Eager execution is a powerful execution environment that evaluates operations immediately. It does not build graphs, and the operations return actual values instead of computational graphs to run later. With Eager execution, TensorFlow calculates the values of tensors as they occur in your code.

In [5]:
# In Tensorflow 2.0, eager execution is enabled by default.
tf.executing_eagerly()

True

In [10]:
x = [[2.]]
m = tf.matmul(x,x)
print("hello, {}".format(m))     #Return results immediately 

hello, [[4.]]


In TensorFlow1.x, the above code will return:

      hello, Tensor("MatMul:0", shape=(1, 1), dtype=float32)

### Eager execution

<font color= red>Easy-to-build\&test</font>

Eager execution is easy to implement, and simplify the model building experience. Advantages of eager execution:

- **An intuitive interface** with natural Python code and data structures;
- **Easier debugging** with calling operations directly to inspect and test models;
- **Natural control flow** with Python, instead of graph control flow; and
- Support for **GPU & TPU acceleration**.

### Graph execution

<font color= red>Efficient and fast</font>

Eager execution is slower than graph execution since eager execution runs all operations one-by-one in Python, it cannot take advantage of potential acceleration opportunities.

Graphs are easy-to-optimize, simplify arithmetic operations. In graph execution, evaluation of all the operations happens only after we've called our program entirely. So, graph execution is
- **Very Fast**
- **Very Flexible**
- **Runs in parallel**, even in sub-operation level;
- **Very efficient**, on multiple devices
- with **GPU & TPU acceleration** capability

Graph execution is ideal for large model training. For small model training, beginners, eager execution is better suited.

### Why TensorFlow adopted Eager Execution?

- Before version 2.0, TensorFlow prioritized graph execution because it was fast, effiicent, and flexible. But it is difficult to implement.
- PyTorch adopted a different approach and prioritized dynamic computation graphs, which is a similar concept to eager execution. Although they are not that efficient, it is intuitive and easy to implement. Then PyTorch became attractive for the newcommers.
- TensorFlow felt threatened, so the TensorFlow team adopted eager execution as the default execution method, and graph execution is optional.

In TensorFlow 2.0, you can decorate a Python function using tf.function() to run it as a single graph object. With this new method, you can easily build models and gain all the graph execution benefits.

Note: TensorFlow2.x, no session, no initialization

In [6]:
import timeit

In [20]:
def eager_function(x):
    result = x**2
    print(result)
    return result

x = tf.constant([1.0,2.0,3.0,4.0,5.0])

graph_function = tf.function(eager_function)  #run the same function with graph execution

In [15]:
### eager execution
eager_function(x)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 1.,  4.,  9., 16., 25.], dtype=float32)>

In [16]:
### graph execution
graph_function(x)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 1.,  4.,  9., 16., 25.], dtype=float32)>

In [21]:
### compare the execution times
print("Eager time:", timeit.timeit(lambda: eager_function(x), number=1))
print("Graph time:", timeit.timeit(lambda: graph_function(x), number=1))

tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
Eager time: 0.000585285946726799
Tensor("pow:0", shape=(5,), dtype=float32)
Graph time: 0.006282627582550049


For simple operations, graph execution does not always perform well because it has to spend the initial computing power to build a graph. If the code is ran 100 times, the results will change dramatically.

In [22]:
### compare the execution times
print("Eager time:", timeit.timeit(lambda: eager_function(x), number=100))
print("Graph time:", timeit.timeit(lambda: graph_function(x), number=100))

tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(5,), dtype=float32)
tf.Tensor([ 1.  4.  9. 16. 25.], shape=(

Let's build a dummy neural network to compare the performances of eager and graph executions. No need to worry about the process of building the model.

In [24]:
# TensorFlow imports
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Flatten, Dense

# Model building
inputs = Input(shape=(28, 28)) 
x = Flatten()(inputs) 
x = Dense(256, "relu")(x)
x = Dense(256, "relu")(x) 
x = Dense(256, "relu")(x) 
outputs = Dense(10, "softmax")(x) 

input_data = tf.random.uniform([100, 28, 28])

# Eager Execution
eager_model = Model(inputs=inputs, outputs=outputs)
print("Eager time:", timeit.timeit(lambda: eager_model(input_data), number=10000))

#Graph Execution 
graph_model = tf.function(eager_model) # Wrap the model with tf.function 
print("Graph time:", timeit.timeit(lambda: graph_model(input_data), number=10000))

Eager time: 18.297894755378366
Graph time: 7.456172604113817


# Tensors

Tensors are TensorFlow's multi-dimensional arrays with uniform type. They are ***immutable***.

[Ref](https://towardsdatascience.com/mastering-tensorflow-tensors-in-5-easy-steps-35f21998bb86)
- variables
- constant
- Placeholder
- SparseTensor

## Create tensors

In [27]:
t1 = tf.constant([[1,2,3,4,5]])
t2 = tf.ones((1,5))
t3 = tf.zeros((1,5))
t4 = tf.range(start=1,limit=6,delta=1)
print(t1)
print(t2)
print(t3)
print(t4)

tf.Tensor([[1 2 3 4 5]], shape=(1, 5), dtype=int32)
tf.Tensor([[1. 1. 1. 1. 1.]], shape=(1, 5), dtype=float32)
tf.Tensor([[0. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


In [66]:
## ragged tensor
ragged_tensor = tf.ragged.constant([[1, 2, 3],[4, 5],[6]])
print(ragged_tensor)
## string tensor
string_tensor = tf.constant(["element1",
                             "element2",
                             "element3"])
print(string_tensor)
## sparse tensor
## when you have holes in your data, Sparse Tensors are to-go objects.
sparse_tensor = tf.sparse.SparseTensor(indices=[[0,0],[2,2],[4,4]],
                                      values = [25,50,100],
                                      dense_shape=[5,5])
print(tf.sparse.to_dense(sparse_tensor))

<tf.RaggedTensor [[1, 2, 3], [4, 5], [6]]>
tf.Tensor([b'element1' b'element2' b'element3'], shape=(3,), dtype=string)
tf.Tensor(
[[ 25   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0  50   0   0]
 [  0   0   0   0   0]
 [  0   0   0   0 100]], shape=(5, 5), dtype=int32)


Attribute of tensors:
- `.ndim` or `tf.rank(...).numpy()`
- `.shape`
    - `tf.reshape(tensor1,[...])`
- `tf.size(...).numpy()`
- `.dtype`

In [36]:
rank_3_tensor = tf.constant([[[0,1,2],[3,4,5]],[[6,7,8],[9,10,11]]])
print(rank_3_tensor)
## rank
print(rank_3_tensor.ndim)
print(tf.rank(rank_3_tensor).numpy())
## shape
print(rank_3_tensor.shape)
## size
print(tf.size(rank_3_tensor).numpy())
## dtype
print(rank_3_tensor.dtype)

tf.Tensor(
[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]], shape=(2, 2, 3), dtype=int32)
3
3
(2, 2, 3)
12
<dtype: 'int32'>


In [60]:
## reshape
reshape_tensor1 = tf.reshape(rank_3_tensor,[2,3,2])
print(reshape_tensor1)
reshape_tensor2 = tf.reshape(rank_3_tensor,[2,-1])
print(reshape_tensor2)
reshape_tensor3 = tf.reshape(rank_3_tensor,[-1])  #flatten the tensor
print(reshape_tensor3)

tf.Tensor(
[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]], shape=(2, 3, 2), dtype=int32)
tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]], shape=(2, 6), dtype=int32)
tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)


## indexing

In [44]:
rank_1_tensor = tf.constant([0,1,2,3,4,5,6,7,8,9,10,11])
print(rank_1_tensor)
## only values
print(rank_1_tensor.numpy())
## indexing
print(rank_1_tensor[0].numpy())
print(rank_1_tensor[-1].numpy())
print(rank_1_tensor[1:-1].numpy())

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11], shape=(12,), dtype=int32)
[ 0  1  2  3  4  5  6  7  8  9 10 11]
0
11
[ 1  2  3  4  5  6  7  8  9 10]


In [45]:
rank_2_tensor = tf.constant([[0,1,2,3,4,5],[6,7,8,9,10,11]])
print(rank_2_tensor)
## indexing
print(rank_2_tensor[0].numpy())
print(rank_2_tensor[1].numpy())
print(rank_2_tensor[0,0].numpy())
print(rank_2_tensor[0,2].numpy())

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]], shape=(2, 6), dtype=int32)
[0 1 2 3 4 5]
[ 6  7  8  9 10 11]
0
2


## Operations with tensors

In [117]:
a = tf.constant([[2, 4], 
                 [6, 8]], dtype=tf.float32)   #if we didn't specify the dtype, it would be tf.int32
b = tf.constant([[1, 3], 
                 [5, 7]], dtype=tf.float32)

In [118]:
## addition
add_tensors = tf.add(a,b)    #the same with a+b 
print(add_tensors)

tf.Tensor(
[[ 3.  7.]
 [11. 15.]], shape=(2, 2), dtype=float32)


In [120]:
## element-wise multiplication
multiply_tensors = tf.multiply(a,b)    #the same with a*b
print(multiply_tensors)

tf.Tensor(
[[ 2. 12.]
 [30. 56.]], shape=(2, 2), dtype=float32)


In [53]:
## matrix multiplication
matmul_tensors = tf.matmul(a,b)
print(matmul_tensors)

tf.Tensor(
[[22. 34.]
 [46. 74.]], shape=(2, 2), dtype=float32)


In [55]:
## max value
print(tf.reduce_max(b).numpy())

7.0


In [58]:
## sum
print(tf.reduce_sum(b).numpy())

16.0


In [56]:
## argmx
print(tf.argmax(b).numpy())

[1 1]


In [57]:
## in softmax function
print(tf.nn.softmax(b).numpy())

[[0.11920291 0.880797  ]
 [0.11920291 0.880797  ]]


In [61]:
m = tf.constant([5])
n = tf.constant([[1,2],[3,4]])

print(tf.multiply(m, n))

tf.Tensor(
[[ 5 10]
 [15 20]], shape=(2, 2), dtype=int32)


# Variables

A TensorFlow Variable is the preferred object type representing a shared and persistent state that you can ***manipulate with any operation***, including TensorFlow models.

In [106]:
## type1: pass a tf.constant()
a = tf.constant([[0.0,1.0],[2.0,3.0]])
print(a)
var_a = tf.Variable(a)
print(var_a)
## type2: pass a single integer
var_b = tf.Variable(10000)
print(var_b)
## type3: pass a list
var_c = tf.Variable([[0.0,1.0],[2.0,3.0]])
print(var_c)
print(var_a == var_c)
## type4: pass a single string
var_d = tf.Variable('string')
print(var_d)
## type5: pass a list of strings
var_e = tf.Variable(['string1','string2'])
print(var_e)

tf.Tensor(
[[0. 1.]
 [2. 3.]], shape=(2, 2), dtype=float32)
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 1.],
       [2., 3.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=10000>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 1.],
       [2., 3.]], dtype=float32)>
tf.Tensor(
[[ True  True]
 [ True  True]], shape=(2, 2), dtype=bool)
<tf.Variable 'Variable:0' shape=() dtype=string, numpy=b'string'>
<tf.Variable 'Variable:0' shape=(2,) dtype=string, numpy=array([b'string1', b'string2'], dtype=object)>


Variable objects are built on top of Tensor objects.

Attribute of variables:
- `.shape`
    - `tf.reshape(var1,[...])`
- `tf.size(...).numpy()`
- `tf.rank(...).numpy()`
- `.dtype`
- `.value()`      
     - every variable must specify its initial value
- `.name`
     - If you don't specify a `name`, TensorFlow assigns a default name
- `.trainable`
- `.device`

In [90]:
print(var_a)
print(var_a.shape)
print(tf.size(var_a).numpy())
print(tf.rank(var_a).numpy())

## The values stroed in the variables 
print(var_a.value())
## The values stroed in the variables as NumPy object
print(var_a.numpy())

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0., 1.],
       [2., 3.]], dtype=float32)>
(2, 2)
4
2
tf.Tensor(
[[0. 1.]
 [2. 3.]], shape=(2, 2), dtype=float32)
[[0. 1.]
 [2. 3.]]


In [107]:
reshape_a1 = tf.reshape(var_a,[4,1])
print(reshape_a1)
reshape_a2 = tf.reshape(var_a,[4,-1])
print(reshape_a2)

tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]], shape=(4, 1), dtype=float32)
tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]], shape=(4, 1), dtype=float32)


In [93]:
## name

var1 = tf.Variable([[0.0,1.0],[2.0,3.0]])
print(var1.name)
var2 = tf.Variable([[0.0,1.0],[2.0,3.0]],name='myname')
print(var2.name)

print(var_a.trainable)
print(var_a.device)

Variable:0
myname:0
True
/job:localhost/replica:0/task:0/device:CPU:0


## Operations with variables

In [99]:
var_a = tf.Variable([[1.0,2.0],[3.0,4.0]])
print('Plus')
print(var_a + 2)
print('\nMinus')
print(var_a - 2)
print('\nMultiply')
print(var_a * 2)
print('\nDivide')
print(var_a / 2)
print('\nMatrix Multiply')
print(var_a @ var_a)   #matrix multipllication
print('\nModulo')
print(var_a % 2)       #modulo

Plus
tf.Tensor(
[[3. 4.]
 [5. 6.]], shape=(2, 2), dtype=float32)

Minus
tf.Tensor(
[[-1.  0.]
 [ 1.  2.]], shape=(2, 2), dtype=float32)

Multiply
tf.Tensor(
[[2. 4.]
 [6. 8.]], shape=(2, 2), dtype=float32)

Divide
tf.Tensor(
[[0.5 1. ]
 [1.5 2. ]], shape=(2, 2), dtype=float32)

Matrix Multiply
tf.Tensor(
[[ 7. 10.]
 [15. 22.]], shape=(2, 2), dtype=float32)

Modulo
tf.Tensor(
[[1. 0.]
 [1. 0.]], shape=(2, 2), dtype=float32)


## Extend

In [110]:
## assign
var_a = tf.Variable([[1.0,2.0],[3.0,4.0]])
var_a.assign([[100,200],[300,400]])
print(var_a)
print(var_a.value())

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[100., 200.],
       [300., 400.]], dtype=float32)>
tf.Tensor(
[[100. 200.]
 [300. 400.]], shape=(2, 2), dtype=float32)


In [112]:
var_a.assign_add([[1,2],[3,4]])
print(var_a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[101., 202.],
       [303., 404.]], dtype=float32)>


In [105]:
## indexing
print(var_a[0].numpy())
print(var_a[1].numpy())
print(var_a[0,0].numpy())
print(var_a[0,1].numpy())
## broadcasting
var_b = tf.Variable([5])
var_c = tf.Variable([[1,2],[3,4]])
print(var_b * var_c)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[100., 200.],
       [300., 400.]], dtype=float32)>
tf.Tensor(
[[100. 200.]
 [300. 400.]], shape=(2, 2), dtype=float32)
[100. 200.]
[300. 400.]
100.0
200.0
tf.Tensor(
[[ 5 10]
 [15 20]], shape=(2, 2), dtype=int32)


## Hardware selection for variables

In [108]:
## Print what type of device our variable is processed with
print(var_a.device)

/job:localhost/replica:0/task:0/device:CPU:0


In [109]:
## tf.device: Set the device we want a particular calculation to be processed with
with tf.device('CPU:0'):
    a = tf.Variable([[1.0,2.0,3.0],[4.0,5.0,6.0]])
    b = tf.Variable([[1.0,2.0,3.0]])
    print(a.device)
    print(b.device)
    
with tf.device('GPU:0'):
    k = a * b
    print(k.device)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:CPU:0


# function

In [19]:
@tf.function
def example():
    a = tf.constant(0.)
    b = 2 * a **2
    return tf.gradients(a + b, [a, b], stop_gradients=[a, b])

example()

[<tf.Tensor 'PartitionedCall:0' shape=() dtype=float32>,
 <tf.Tensor 'PartitionedCall:1' shape=() dtype=float32>]

# Some common seen command in TensorFlow1.x

In [ ]:
tf.compat.v1.disable_eager_execution()
placeholder

# session

In [ ]:
# work/DNN/scratch

#work/DNN/ODE-tf/tf2 vs tf1 (eg from stack)

In [10]:
with tf.compat.v1.Session() as sess:
    x = tf.constant([1., 2.])
    y = x**2
    tf.compat.v1.global_variables_initializer().run()
    print(y.eval())

[1. 4.]


In [11]:
with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)
    x = tf.constant([1., 2.])
    y = x**2
    print(y.eval())

[1. 4.]


In [18]:
tf.compat.v1.disable_eager_execution()
x = tf.compat.v1.placeholder(tf.float64,[None,1])
y = x**2

init = tf.compat.v1.global_variables_initializer()
with tf.compat.v1.Session() as sess:
    sess.run(init)
    a = np.array([[1],[2]])   #cannot be tf.constant
    c = sess.run(y,feed_dict={x:a})    #print(c) not print(y)
    print(c)

[[1.]
 [4.]]


## gradient

In [ ]:
# tf.gradients

x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
    x_sq = x * x
    with t.stop_recording():
        y_sq = y * y
    z = x_sq + y_sq

grad = tf.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

multiple tapes

In [ ]:
#work/python/keras-1

x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
    tape0.watch(x0)
    tape1.watch(x1)

    y0 = tf.math.sin(x0)
    y1 = tf.nn.sigmoid(x1)

    y = y0 + y1

    ys = tf.reduce_sum(y)

In [ ]:
print(tape0.gradient(ys, x0).numpy())  # cos(x) => 1.0

print(tape1.gradient(ys, x1).numpy())   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25

higher-order gradient

In [ ]:
#work/python/keras-1

x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
    with tf.GradientTape() as t1:
        y = x * x * x

    # Compute the gradient inside the outer `t2` context manager
    # which means the gradient computation is differentiable as well.
    dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

In [ ]:
tf.compat.v1.squeeze

In [ ]:
tf.dynamic_stitch()

In [ ]:
tf.slice    #work/DNN/scratch

In [ ]:
tf.compat.v1.get_variable     #work/DNN/scratch

var2 = tf.compat.v1.get_variable(initializer=tf.constant_initializer(3.), dtype=tf.float64, name="var1", shape=[2,3])

In [ ]:
tf.math.square(2)    #work/DNN/scratch

def mysquare(x):
    return tf.math.square(x)

with tf.compat.v1.Session() as sess:
    xx = tf.constant([1., 2.])
    yy = mysquare(xx)
    tf.compat.v1.global_variables_initializer().run()

    print(xx.eval(), yy.eval())